In [ ]:
import pandas as pd
from utilities.data_manager import DataManager
from utilities.backtest_analysis import BacktestAnalysis
from strategies import envelope as strat

In [ ]:
symbol = "BTC/USDT:USDT"
data = DataManager(name="bitget")
ohlcv = data.load(symbol, timeframe="1h", start_date="2021-01-02 00:00:00")

In [ ]:
strategy_params = {
    'average_type': 'DCM',  # 'DCM', 'SMA', 'EMA', 'WMA',
    'average_period': 6,
    'envelopes': [0.07, 0.11, 0.14],
    'stop_loss_pct': 0.4,
    'price_jump_pct': 0.15,  # optional
    'position_size_percentage': 100,  #  % of the balance spread equally across each envelope
    # 'position_size_fixed_amount': 100, # fixed amount spread equally across each envelope
    # 'mode': "long", "short", "both" (default)
}
strategy = strat.Strategy(strategy_params, ohlcv)
strategy.run_backtest(initial_balance=1000, leverage=1, open_fee_rate=0.0002, close_fee_rate=0.0006)

In [ ]:
strategy.trades_info
# path = symbol.replace('/', '-').replace(':', '-') 
# strategy.save_trades_info(path)

In [ ]:
strategy.equity_record
# strategy.save_equity_record(path)

In [ ]:
results = BacktestAnalysis(strategy)
results.print_metrics()
# path = "..."
# results.print_metrics(path)

In [ ]:
results.plot_equity()
# results.plot_equity(path)

results.plot_drawdown()
# results.plot_drawdown(path)

In [ ]:
results.plot_monthly_performance()
# results.plot_monthly_performance(path)

In [ ]:
indicators = {
    'average': {
        'color': "white",
        'df': pd.DataFrame({
            'time': strategy.data.index,
            'average': strategy.data['average'],
        }).dropna()        
    } 
}

for i, e in enumerate(strategy_params['envelopes']):
    indicators.update({
        f'band_high_{i+1}' : {
            'color': "red",
            'df': pd.DataFrame({
                'time': strategy.data.index,
                f'band_high_{i+1}': strategy.data[f'band_high_{i+1}'],
            }).dropna()   
        },
        f'band_low_{i+1}' : {
            'color': "green",
            'df': pd.DataFrame({
                'time': strategy.data.index,
                f'band_low_{i+1}': strategy.data[f'band_low_{i+1}'],
            }).dropna()  
        },
    })

results.plot_candlestick(indicators=indicators)